In [ ]:
# Adapted from
# https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a


In [32]:
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# Libraries to import and process the data set
from Utilities import directories
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

In [45]:
### --------- Load and transform data ---------- ###
with open(directories.data + "/example_extract.txt", "rb") as file:   # Unpickling
    data = pickle.load(file)

# Transfer data to pandas ds and set model output (classes) as categorical
data_df = pd.DataFrame(data, index=['Description', 'CPC_label']).transpose()
data_df['CPC_label'] = pd.Categorical(data_df['CPC_label'])
# Transform output to numeric
data_df['CPC_code'] = data_df['CPC_label'].cat.codes
# Drop rows with classes that only occur once
data_df.CPC_code.value_counts()
# Split into train and test
train, test = train_test_split(data_df, test_size = 0.2)
print(len(train), len(test))

38 10


In [ ]:
### --------- Setup BERT ---------- ###
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
# from_pretrained loads weights from pretrained model
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [21]:
# Vie default configurations
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [46]:
### ------- Build the model ------- ###
# Load the MainLayer
bert = transformer_model.layers[0]

# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
cpc_class = Dense(units=len(data_df.CPC_code.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='cpc')(pooled_output)
outputs = {'cpc': cpc_class}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
cpc (Dense)                  (None, 16)                12304     
Total params: 109,494,544
Trainable params: 109,494,544
Non-trainable params: 0
_________________________________________________________________


In [47]:
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'cpc': CategoricalCrossentropy(from_logits = True)}
metric = {'cpc': CategoricalAccuracy('accuracy')}# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)

# Ready output data for the model
y_cpc = to_categorical(data_df['CPC_code'])

# Tokenize the input (takes some time)
x = tokenizer(
    text=data_df['Description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'cpc': y_cpc},
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 2.8358 - accuracy: 0.0526 - val_loss: 2.6805 - val_accuracy: 0.1000
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 2.7080 - accuracy: 0.0000e+00 - val_loss: 2.5176 - val_accuracy: 0.2000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 2.4685 - accuracy: 0.1053 - val_loss: 2.3797 - val_accuracy: 0.4000
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 2.3680 - accuracy: 0.5000 - val_loss: 2.2693 - val_accuracy: 0.6000
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 2.2559 - accuracy: 0.4474 - val_loss: 2.2116 - val_accuracy: 0.6000
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 2.1678 - accuracy: 0.4474 - val_loss: 2.1446 - val_accuracy: 0.